# Battery Arbitrage Optimization

## Solution Overview

The task is to optimize revenue generation from performing of energy arbitrage. The approach used in this solution will follow maximizing of profit that is given as a difference between revenue and cost. It is assumed that both revenue and cost only arise from settlements on day ahead market, hence neglecting any further incurred costs (e.g. distribution, fees), as well as other CapEx and OpEx costs. The objective function will thus be given by:

\begin{equation}
\begin{array}{rrcll}
\displaystyle \max_{P_{in}, P_{out}} & NP \\
\textrm{s.t.} & NP & = & \displaystyle  \sum_{t=0}^{t_{max}} c(t) * \left( \frac{P_{in} (t)}{\mathrm{eff}_{in}} - P_{out} (t) * \mathrm{eff}_{out} \right) & \mathrm{\left( con1 \right)}\\
\end{array}
\end{equation}

where $NP$ denotes profit/net profit, $P_{in}, P_{out}$ denote power charged and discharged respectively with their corresponding efficiencies $\mathrm{eff}_{in}, \mathrm{eff}_{out}$ and $c(t)$ vector of costs. The variables are indexed by timestamps $t$. Note, that although $\mathrm{eff}_{in}, \mathrm{eff}_{out}$ are not given in the problem, I believe at least one of the pair is needed, so as to deduce the other using round-trip efficiency $\mathrm{eff}_{rt}$ from $\mathrm{eff}_{rt} = \mathrm{eff}_{in} * \mathrm{eff}_{out}$

Charging and discharging is limited by:

\begin{equation}
\begin{array}{rrcl}
& P_{in} & <= & \displaystyle  C(t) * P_{in.max}  & \mathrm{\left( con2 \right)}\\
& P_{out} & <= & \displaystyle  D(t) * P_{out.max}  & \mathrm{\left( con3 \right)}\\
\end{array}
\end{equation}

where C and D are binary indicators of whether charging or discharging occurs at $t$, and $P_{in.max}, P_{out.max}$ are maximum charging and discharging limits. Daily discharged energy is also limited by:

\begin{equation}
\begin{array}{rrcl}
& \displaystyle  \sum_{t=0}^{t_{max}} T * P_{out}(t) & <= & W_{out.max} & \mathrm{\left( con4 \right)}\\
\end{array}
\end{equation}

where T is period length of time resolution used in the model and $W_{out.max}$ is the daily energy discharge limit. 

Even though this is an idealized scenario, one does not simply escape almighty energy conservation law. This is expressed through its effect on state of charge (soc) as:

\begin{equation}
\begin{array}{rrcl}
& \mathrm{soc}(t) & = & \mathrm{soc}(t - 1) + T * \frac{P_{in}(t) - P_{out}(t)}{B_{cap}} & \mathrm{\left( con5 \right)}\\
\end{array}
\end{equation}

with $B_{cap}$ used to denote battery capacity. $\mathrm{soc}$ is initialized with its initial value. Lastly, a restriction is imposed to prevent both charging and discharging occurring at the same time:

\begin{equation}
\begin{array}{rrcl}
& C(t) + D(t) & <= & 1 & \mathrm{\left( con6 \right)}\\
\end{array}
\end{equation}

Domains of the variables are as follows:

\begin{equation}
\begin{array}{rcccl}
C(t), D(t) & \in & \{0, 1\} \\
1 - \mathrm{dod}_{max} & <= & \mathrm{soc}(t) & <= & 1 \\
P_{in}, P_{out} & >= & 0 \\
NP & \in & \mathbb{R} \\
\end{array}
\end{equation}

where $\mathrm{dod}_{max}$ is the (maximum) depth of discharge.

The formulation can be solved using a MILP solver. In this example, the solution is performed on a set of $$ t \in \{0, 1, 2 ..., 23\} $$ for a single selected day, leading to a computationally inexpensive solution. Note, that with added complexity, the problem could become significantly more expensive. One of the ways to obtain solution in real time then could be to drop $C(t), D(t)$ and handle the $\mathrm{\left( con6 \right)}$ constraint in postprocessing (i.e. if both charging and discharging occurs, one can take the difference of the two as indication of how much energy is to be charged/discharged.

Vector of costs $c(t)$ is obtained as a vector of LBMP values for the required zone (N.Y.C.) averaged over the size of bins (given by the required resolution).

As for further inputs to improve the optimization, I would suggest supplying one of the already mentioned pair $\mathrm{eff}_{in}, \mathrm{eff}_{out}$ on top of $\mathrm{eff}_{rt}$. Furthermore, the model can be made more realistic by considering physical phenomena such as reaction/ramp-up time, business realities such as other associated costs etc. A solution with higher average gain per day can be obtained by widening the scope of the problem to use the optimization on horizon greater than one day. Some other constraints are often provided regarding scheduling, such as minimum time between charging and discharging, minimum uninterrupted runtime or threshold price over which/under which the system does not buy/sell.

The following code solves the problem using pyomo and glpk solver. The required results are summarized in the end of the notebook. 

## Code

In [1]:
import datetime
import time
import os

import numpy as np
import pyomo.environ as pyo

import expressions
from utils import get_prices, sample_to_time

### Inputs

In [2]:
# data-specific inputs
resolution = 0.5  # hours
data_args = {
    'req_date': datetime.datetime(2022, 8, 6),
    'data_dir': 'data/',
    'zone': 'N.Y.C.',
    'resolution': resolution,
    'exp_date_format': '%Y%m%d',
    'date_col': 'Time Stamp',
    'zone_col': 'Name',
    'cost_col': 'LBMP'
}

In [3]:
# battery asset properties
max_charge_power = 100  # kW
max_discharge_power = 100  # kW
discharge_ener_cap = 200  # kWh
max_daily_discharge = 200  # kWh
depth_of_discharge = 1  # fraction
state_of_charge_init = 0  # fraction
eff_rt = 0.85  # fraction
eff_in = 0.9  # fraction

In [4]:
# default and derived ipupts
solver = 'glpk'
time_frame_length = 24  # hours
min_daily_sell_volume = 0  # kWh
min_daily_buy_volume = 0  # kWh
min_charge_power = 0  # kW
min_discharge_power = 0  # kW
time_samples = int(time_frame_length / resolution)
eff_out = eff_rt / eff_in

In [5]:
model = pyo.ConcreteModel(
    name='battery model'
)

### Sets

In [6]:
model.samples = pyo.Set(
    initialize=np.arange(time_samples), 
    doc='time samples'
)

### Parameters

#### Costs

In [7]:
c = get_prices(**data_args)
# the costs need to be adjusted to $/kWh basis
c /= 1e3
c = c.rename('LBMP in $/kWh')
model.c = pyo.Param(
    model.samples, 
    initialize=c.to_numpy(),
    doc='energy costs'
)

#### Misc

In [8]:
model.dec = pyo.Param(
    initialize=discharge_ener_cap,
    doc='discharge/battery energy capacity (kWh)'
)
model.eff_in = pyo.Param(
    initialize=eff_in,
    doc='AC to DC charging efficiency (fraction)'
)
model.eff_out = pyo.Param(
    initialize=eff_out,
    doc='DC to AC discharging efficiency (fraction)'
)
model.mcp = pyo.Param(
    initialize=max_charge_power,
    doc='max charging power (kW)'
)
model.mdp = pyo.Param(
    initialize=max_discharge_power,
    doc='max discharging power (kW)'
)
model.mdd = pyo.Param(
    initialize=max_daily_discharge,
    doc='max daily discharged energy (kWh)'
)
model.res = pyo.Param(
    initialize=resolution,
    doc='model resolution (h)'
)
model.soc_init = pyo.Param(
    initialize=state_of_charge_init,
    doc='initial battery state of charge (fraction)'
)

### Variables

In [9]:
model.state_of_charge = pyo.Var(
    model.samples, 
    bounds=(1 - depth_of_discharge, 1), 
    initialize=0,
    doc='battery state of charge (fraction)'
)

In [10]:
model.power_out = pyo.Var(
    model.samples, 
    bounds=(min_discharge_power, max_discharge_power),
    doc='discharged power (kW)'
)

In [11]:
model.power_in = pyo.Var(
    model.samples, 
    bounds=(min_charge_power, max_charge_power),
    doc='charged power (kW)'
)

In [12]:
model.profit = pyo.Var(
    within=pyo.Reals,
    doc='total profit ($)'
)

In [13]:
model.is_charging = pyo.Var(
    model.samples, 
    within=pyo.Binary,
    doc='charging indicator (0/1)'
)

In [14]:
model.is_discharging = pyo.Var(
    model.samples, 
    within=pyo.Binary,
    doc='discharging indicator (0/1)'
)

### Constraints

In [15]:
model.con1 = pyo.Constraint(
    rule=expressions.objective,
    doc='profit calculation'
)

In [16]:
model.con2 = pyo.Constraint(
    model.samples,
    rule=expressions.charge_limit,
    doc='enforce maximum charging power limit'
)

In [17]:
model.con3 = pyo.Constraint(
    model.samples,
    rule=expressions.discharge_limit,
    doc='enforce maximum discharging power limit'
)

In [18]:
model.con4 = pyo.Constraint(
    expr=expressions.daily_discharge_limit,
    doc='enforce daily discharge limit'
)

In [19]:
model.con5 = pyo.Constraint(
    model.samples, 
    rule=expressions.energy_balance,
    doc='enforce energy balance'
)

In [20]:
model.con6 = pyo.Constraint(
    model.samples,
    rule=expressions.only_one_op,
    doc='prevent charging and discharging at the same time'
)

### Objective

In [21]:
model.obj = pyo.Objective(
    expr=model.profit, 
    sense=pyo.maximize,
    doc='maximize profit'
)

### Solution

In [22]:
%%time
solver = pyo.SolverFactory(solver)
res = solver.solve(model)

CPU times: total: 46.9 ms
Wall time: 89.6 ms


## Results

In [23]:
# results summary
res.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 123.336601851852
  Upper bound: 123.336601851852
  Number of objectives: 1
  Number of constraints: 195
  Number of variables: 242
  Number of nonzeros: 625
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04356527328491211
# ----------------------------------------------------------
#   Solution Information
# ---------

In [24]:
revenue = pyo.value(sum(
    model.power_out[t] * eff_out * c[t] for t in model.samples))
cost = pyo.value(sum(
    model.power_in[t] / eff_in * c[t] for t in model.samples))
profit = pyo.value(model.profit)
print('Revenue/turnover for the day is ${:.2f}.'.format(revenue))
print('Costs for the day are ${:.2f}.'.format(cost))
print('Profit for the day is ${:.2f}.'.format(profit))

Revenue/turnover for the day is $151.31.
Costs for the day are $27.97.
Profit for the day is $123.34.


In [25]:
print('LBMP at each timestamp:')
print(c)

LBMP at each timestamp:
Time Stamp
2022-08-06 00:30:00    0.094713
2022-08-06 01:00:00    0.089380
2022-08-06 01:30:00    0.081862
2022-08-06 02:00:00    0.078428
2022-08-06 02:30:00    0.089842
2022-08-06 03:00:00    0.085545
2022-08-06 03:30:00    0.087460
2022-08-06 04:00:00    0.083198
2022-08-06 04:30:00    0.084745
2022-08-06 05:00:00    0.081065
2022-08-06 05:30:00    0.085153
2022-08-06 06:00:00    0.080140
2022-08-06 06:30:00    0.064920
2022-08-06 07:00:00    0.071532
2022-08-06 07:30:00    0.061592
2022-08-06 08:00:00    0.061438
2022-08-06 08:30:00    0.063818
2022-08-06 09:00:00    0.066528
2022-08-06 09:30:00    0.081670
2022-08-06 10:00:00    0.100718
2022-08-06 10:30:00    0.100278
2022-08-06 11:00:00    0.105920
2022-08-06 11:30:00    0.123553
2022-08-06 12:00:00    0.123640
2022-08-06 12:30:00    0.124618
2022-08-06 13:00:00    0.129742
2022-08-06 13:30:00    0.130053
2022-08-06 14:00:00    0.153170
2022-08-06 14:30:00    0.158213
2022-08-06 15:00:00    0.164915
2022-

In [26]:
print('Revenue/Cost over the day:')
for t in model.samples:
    ts = sample_to_time(t, resolution)
    cost = pyo.value(model.power_in[t]) / eff_in * c[t]
    revenue = pyo.value(model.power_out[t]) * eff_out * c[t]
    print(ts, '${:.2f}'.format(revenue - cost))

Revenue/Cost over the day:
00:30:00 $0.00
01:00:00 $0.00
01:30:00 $0.00
02:00:00 $0.00
02:30:00 $0.00
03:00:00 $0.00
03:30:00 $0.00
04:00:00 $0.00
04:30:00 $0.00
05:00:00 $0.00
05:30:00 $0.00
06:00:00 $0.00
06:30:00 $-7.21
07:00:00 $0.00
07:30:00 $-6.84
08:00:00 $-6.83
08:30:00 $-7.09
09:00:00 $0.00
09:30:00 $0.00
10:00:00 $0.00
10:30:00 $0.00
11:00:00 $0.00
11:30:00 $0.00
12:00:00 $0.00
12:30:00 $0.00
13:00:00 $0.00
13:30:00 $0.00
14:00:00 $0.00
14:30:00 $0.00
15:00:00 $0.00
15:30:00 $0.00
16:00:00 $0.00
16:30:00 $35.58
17:00:00 $0.00
17:30:00 $21.78
18:00:00 $0.00
18:30:00 $0.00
19:00:00 $40.59
19:30:00 $53.36
20:00:00 $0.00
20:30:00 $0.00
21:00:00 $0.00
21:30:00 $0.00
22:00:00 $0.00
22:30:00 $0.00
23:00:00 $0.00
23:30:00 $0.00
00:00:00 $0.00


In [27]:
discharged_throughput = pyo.value(sum(model.power_out[t] for t in model.samples))
print('Discharged energy throughput for the day is {} kWh'.format(discharged_throughput))

Discharged energy throughput for the day is 400.0 kWh


In [28]:
print('Energy of the battery over the day (kWh):')
for t in model.samples:
    ts = sample_to_time(t, resolution)
    energy_level = pyo.value(model.state_of_charge[t]) * discharge_ener_cap
    print(ts, '{:.2f}'.format(energy_level))

Energy of the battery over the day (kWh):
00:30:00 0.00
01:00:00 0.00
01:30:00 0.00
02:00:00 0.00
02:30:00 0.00
03:00:00 0.00
03:30:00 0.00
04:00:00 0.00
04:30:00 0.00
05:00:00 0.00
05:30:00 0.00
06:00:00 0.00
06:30:00 50.00
07:00:00 50.00
07:30:00 100.00
08:00:00 150.00
08:30:00 200.00
09:00:00 200.00
09:30:00 200.00
10:00:00 200.00
10:30:00 200.00
11:00:00 200.00
11:30:00 200.00
12:00:00 200.00
12:30:00 200.00
13:00:00 200.00
13:30:00 200.00
14:00:00 200.00
14:30:00 200.00
15:00:00 200.00
15:30:00 200.00
16:00:00 200.00
16:30:00 150.00
17:00:00 150.00
17:30:00 100.00
18:00:00 100.00
18:30:00 100.00
19:00:00 50.00
19:30:00 0.00
20:00:00 0.00
20:30:00 0.00
21:00:00 0.00
21:30:00 0.00
22:00:00 0.00
22:30:00 0.00
23:00:00 0.00
23:30:00 0.00
00:00:00 0.00


In [29]:
print('Net power of the battery over the day (kW):')
for t in model.samples:
    ts = sample_to_time(t, resolution)
    net_power = pyo.value(model.power_in[t] - model.power_out[t])
    print(ts, '{:.2f}'.format(net_power))

Net power of the battery over the day (kW):
00:30:00 0.00
01:00:00 0.00
01:30:00 0.00
02:00:00 0.00
02:30:00 0.00
03:00:00 0.00
03:30:00 0.00
04:00:00 0.00
04:30:00 0.00
05:00:00 0.00
05:30:00 0.00
06:00:00 0.00
06:30:00 100.00
07:00:00 0.00
07:30:00 100.00
08:00:00 100.00
08:30:00 100.00
09:00:00 0.00
09:30:00 0.00
10:00:00 0.00
10:30:00 0.00
11:00:00 0.00
11:30:00 0.00
12:00:00 0.00
12:30:00 0.00
13:00:00 0.00
13:30:00 0.00
14:00:00 0.00
14:30:00 0.00
15:00:00 0.00
15:30:00 0.00
16:00:00 0.00
16:30:00 -100.00
17:00:00 0.00
17:30:00 -100.00
18:00:00 0.00
18:30:00 0.00
19:00:00 -100.00
19:30:00 -100.00
20:00:00 0.00
20:30:00 0.00
21:00:00 0.00
21:30:00 0.00
22:00:00 0.00
22:30:00 0.00
23:00:00 0.00
23:30:00 0.00
00:00:00 0.00
